# 기사 URL이 주어졌을 때 본문 발췌하기

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs

1. Daum뉴스의 class='news_view'인 div태그에 본문 내용이 포함되어 있음을 확인
2. 크롤링 과정에서 제거해야 할 정보들 확인
 - 기자의 이메일(당연히 @를 포함하는 문자열로 식별가능)
 - 괄호에 둘러쌓여 부수적인 정보를 제공하는 기타 내용들(신문사 이름 등)
 
이 노트북에서 예시로 한 기사의 본문을 추출해 보려고 한다.

In [2]:
response = requests.get("https://news.v.daum.net/v/20200902223053648")
parsed = bs(response.text, 'html.parser')

참고로 이런 제목의 기사다.

In [3]:
parsed.select('div.head_view h3')[0].get_text()

'김현미 편든 노영민 "청년 분양대책 곧 발표..물량 어마어마"'

기사의 본문 부분(body)에서 내용(text)을 추출하는 코드다.

In [4]:
body = parsed.find('div', attrs={'class':'news_view'})
text = body.find_all('p', attrs={'dmcf-ptype':'general'})

각 태그에서 텍스트를 추출하는 동시에 기자의 이메일을 제거하는 코드다.

In [5]:
extracted = ' '.join([tag.get_text().strip() for tag in text if '@' not in tag.get_text()])

괄호로 싸인 문자열을 제거하는 과정에서 괄호에 싸인 글자 수를 최대 20까지로 지정했다. 글자 수에 제한을 두지 않으면 기사 초반에 있는 여는 괄호가 기사 말미에 있는 닫는 괄호와 한 쌍인 것으로 인식되서 기사 전체가 다 사라지는 불상사가 발생할 수 있는데, 이를 방지하기 글자 수를 지정했다. 무엇보다 20자가 넘어가는 정보는 부수적인 내용이 아닐 가능성이 크다. 결과를 확인하면 아래와 같다.

In [6]:
pattern = re.compile("[\[(].{1,20}[\])]")
pattern.sub('', extracted)

'노영민 대통령 비서실장이 " 청년 분양에 대해 조만간 구체적으로 대책을 발표할 것"이라고 밝혔다. 노 실장은 2일 국회 운영위에서 김성원 국민의힘 의원이 \'주택을 살 수 없다는 30·40 청년의 불만을 청와대가 인식해야 한다\'고 지적하자 "이번 발표는 청년들에게 희망을 줄 수 있을 것"이라며 이같이 말했다. 노 실장은 "2022년까지 수도권 공공택지 분양이 37만호인데 이것에 대해 사전청약과 본청약을 언제 분양할지 조만간 국토부가 발표할 것"이라며 "어마어마한 물량"이라고 강조했다. 그는 이어 "김현미 국토부 장관이 최근 \'30대 청년들 영끌해서 집 사지 말고 분양을 기다리라\'고 한 것이 이것 아닌가 생각한다"고 덧붙였다. 노 실장은 \'새로 매입한 사저 부지에 단독주택이 포함돼 있어 문재인 대통령이 2주택자가 됐다\'는 내용의 곽상도 국민의힘 의원 질의에는 "사저 이전 부지에 건물이 지어지면  양산 집은 처분할 계획"이라고 설명했다. 대통령 취임 후 직접 농사를 지으러 간 적이 있느냐 질문에 노 실장은 "양산에 방문할 때 돌본 것으로 알고 있다"면서도 몇차례였냐는 물음에는 "대통령의 비공개 일정은 모두 국가 1급 비밀이기 때문에 밝힐 수 없다"고 말했다. 곽 의원은 "대통령도 공무원이기 때문에 영리업무를 할 수 없다"며 "농업경영은 겸직금지에 해당한다"고 주장하자 노 실장은 "해당한다고 생각하지 않는다"고 반박했다. 같은 당 박대출 의원이 \'농지인 양산 사저 부지의 형질을 변경하면 재산 가치가 올라가는데 국민은 퇴임 보너스라고 생각할 것\'이라고 지적한 데 대해 노 실장은 "서울 강남 아파트도 아니고 강남에 전임 대통령처럼 그런 게 아니잖느냐"고 언급했다.'